# Estimating Assessed Value of Properties    Using the Zillow Dataset

Cris Giovanoni  
October 21, 2019
___

## Notes  
  
### Assumptions
- Square footage accounted all living, finished spaces. The values **excluded** the following:
  - Basement
  - Garage
  - Yard
  - Entire lot
- Types of Single Unit housing included in the analysis:
   - 261	Single Family Residential
   - 262	Rural Residence
   - 263	Mobile Home
   - 264	Townhouse
   - 265	Cluster Home
   - 268	Row House
   - 269	Planned Unit Development
   - 273	Bungalow
   - 275	Manufactured, Modular, Prefabricated Homes
   - 276	Patio Home

### Workflow
- I. Create baseline model
- II. Create Ω model, going through entire pipeline
- III. Compare baseline model and Ω model using the test data

### Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA

import zillow_util as util

---
## I. Create Baseline Model

Get data from SQL and filter data with the following conditions:
tk


In [2]:
zillow = util.filter_zillow_baseline(util.get_sql_zillow())

Separate the features (X) and target (y) columns. Then, split data to train and test, resulting to four data frames with X and y separated and partitioned into two.

In [3]:
X = zillow[["sqft","bathcnt","bedcnt"]]
y = zillow.propvalue

In [4]:
X_train, X_test, y_train, y_test = util.split_my_data(X, y, 0.7)

Scale train and test data with all X.  
Scaled X train data will be fitted into to the model later.  
Scaled X test data will be used post-modelling phase to test the baseline model.

In [5]:
scaled_X_train, scaled_X_test, standard_scaler = util.standard_scaler(X_train, X_test)

Create a linear model with train data, i.e., scaled X data and unscaled y data.

In [6]:
lm_base, lm_base_intercept, lm_base_coeff = util.generate_linear_model(scaled_X_train, y_train)

In [7]:
print(lm_base)
print(lm_base_intercept)
print(lm_base_coeff)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
528745.8490634661
[ 422257.73320622   73656.54782346 -136593.73861784]


---
## II. Create  Ω Model

## PROJECT PLAN
**PROJECT PLAN** -> ACQUIRE -> PRE-PROCESS -> EXPLORE -> MODEL

PROJECT PLANNING & README
Brainstorming ideas, hypotheses, related to how variables might impact or relate to each other, both within independent variables and between the independent variables and dependent variable, and also related to any ideas for new features you may have while first looking at the existing variables and challenge ahead of you.

Have a detailed README.md file for anyone who wants to check out your project. In this file should be a description of what the project is, and any instructions necessary for someone else to clone your project and run the code on their own laptop.

"PREDICTIVE POWER"

## ACQUIRE
PROJECT PLAN -> **ACQUIRE** -> PRE-PROCESS -> EXPLORE -> MODEL

Get data from SQL.  
Perform initial cleanup by casting columns to appropriate data types.
  - Re-type transaction date as a date data type. After which, extract properties whose transaction date fall between May and June 2017.
  - Rename columns for efficiency.


In [8]:
zillow = util.filter_zillow(util.get_sql_zillow())

In [9]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15740 entries, 868 to 37865
Data columns (total 6 columns):
id           15740 non-null int64
sqft         15712 non-null float64
bathcnt      15740 non-null float64
bedcnt       15740 non-null float64
transdate    15740 non-null datetime64[ns]
propvalue    15740 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 860.8 KB


## PRE-PROCESS
PROJECT PLAN -> ACQUIRE -> **PRE-PROCESS** -> EXPLORE -> MODEL

### A. Clean Data

Look for missing values.
 - Observations that don't have square footage also don't contain bathroom and bedroom count. Because the values of these features add little to no value to the data, they are dropped.

In [10]:
zillow.isnull().sum()

id            0
sqft         28
bathcnt       0
bedcnt        0
transdate     0
propvalue     0
dtype: int64

In [11]:
nulls = zillow[zillow.sqft.isnull()]
nulls

,id,sqft,bathcnt,bedcnt,transdate,propvalue
22402,14466991,NaN,0.0,0.0,2017-05-01,2493444.0
23075,13972530,NaN,0.0,0.0,2017-05-04,270009.0
23076,14325627,NaN,0.0,0.0,2017-05-04,7074075.0
23095,12686981,NaN,0.0,0.0,2017-05-04,5461875.0
23861,10746614,NaN,0.0,0.0,2017-05-08,230876.0
24796,14175690,NaN,0.0,0.0,2017-05-11,157074.0
24873,12131538,NaN,0.0,0.0,2017-05-11,225126.0
26411,17292027,NaN,0.0,0.0,2017-05-18,3210155.0
27852,14175815,NaN,0.0,0.0,2017-05-24,385795.0
27931,14430787,NaN,0.0,0.0,2017-05-25,5126781.0


In [12]:
zillow = zillow.dropna(subset=['sqft']) #Drop NaNs in the sqft column

Look for duplicates.
 - There are 5 duplicates.
 - All duplicates point to the same property (same sqft, nos. of bathroom and bedroom, and property value); the only difference is the transaction date.
 - Observations with more recent transaction date are kept.

In [13]:
dups = zillow[zillow.duplicated(subset=["id"])]
dups_ids = list(dups.id)

mask = ~zillow.id.isin(dups_ids) #Returns True for non-duplicates
zillow.loc[~mask] #See what the duplicates contain

,id,sqft,bathcnt,bedcnt,transdate,propvalue
23560,14074415,1025.0,1.0,2.0,2017-05-05,48107.0
23561,14074415,1025.0,1.0,2.0,2017-05-12,48107.0
24099,162960529,2459.0,4.0,4.0,2017-05-09,479000.0
24100,162960529,2459.0,4.0,4.0,2017-05-26,479000.0
27348,17280166,1816.0,3.0,4.0,2017-05-23,350701.0
27349,17280166,1816.0,3.0,4.0,2017-06-15,350701.0
27420,14254548,2126.0,2.0,4.0,2017-05-23,98473.0
27421,14254548,2126.0,2.0,4.0,2017-06-12,98473.0
30899,11991059,8469.0,6.0,12.0,2017-06-06,2485282.0
30900,11991059,8469.0,6.0,12.0,2017-06-09,2485282.0


In [14]:
zillow = zillow.sort_values("transdate").drop_duplicates('id',keep='last')
zillow.loc[~mask] #Check if duplicates are indeed dropped

,id,sqft,bathcnt,bedcnt,transdate,propvalue
23561,14074415,1025.0,1.0,2.0,2017-05-12,48107.0
24100,162960529,2459.0,4.0,4.0,2017-05-26,479000.0
27421,14254548,2126.0,2.0,4.0,2017-06-12,98473.0
30901,11991059,8469.0,6.0,12.0,2017-06-13,2485282.0
27349,17280166,1816.0,3.0,4.0,2017-06-15,350701.0


### B. Split-Scale Data

**SPLIT**
1. Separate the features (X) and target (y) columns.
2. Using `split_my_data` function from `zillow.util`, split further to train and test, resulting to four data frames with X and y separated and partitioned into two-train and test.

In [15]:
X = zillow[["sqft","bathcnt","bedcnt"]]
y = zillow.propvalue

In [16]:
X_train, X_test, y_train, y_test = util.split_my_data(X, y, 0.7)

**SCALE**  
1. Scale all feature data, i.e., X_train and X_test, using the `standard_scaler` function from `zillow.util`.
    - _Note:_ Features are scaled to mimic a normal distribution because Pearson's R will be obtained later.
2. Scale X_train data. It will be fitted into to the model later.  
3. Scale X_test data. It will be used post-modelling phase to test Ω model.

In [17]:
scaled_X_train, scaled_X_test, standard_scaler = util.gaussian_scaler(X_train, X_test)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


NameError: name 'uniform_scaler' is not defined

_Note: Until post-modelling, only train data will be used henceforth._

## EXPLORE
PROJECT PLAN -> ACQUIRE -> PRE-PROCESS -> **EXPLORE** -> MODEL

### A. Find Correlations
Between Features to Target, Features to Features

1. Look at Descriptive Stats of train data.
2. Check if square footage, no. of bathrooms, and no. of bedrooms (features, X) are statistically significant with property value (target, y).
3. Find strength of Linear Correlations between features and target through Pearson's R and heatmap plot.

#### DESCRIPTIVE STATISTICS

In [ ]:
#Merge scaled X train and y train data to have a full train dataset
train_combined = scaled_X_train.copy()
train_combined["propvalue"] = y_train

In [ ]:
train_combined.describe()

In [ ]:
grid = sns.pairplot(train_combined, vars=["sqft","bathcnt","bedcnt","propvalue"])

#### CORRELATIONS

In [ ]:
corrsqft, psqft = stats.pearsonr(train_combined.sqft, train_combined.propvalue)
corrbth, pbth = stats.pearsonr(train_combined.bathcnt, train_combined.propvalue)
corrbed, pbed = stats.pearsonr(train_combined.bedcnt, train_combined.propvalue)

pearsonr = [corrsqft, corrbth, corrbed]
pval = [psqft, pbth, pbed]

r_pval = pd.DataFrame({"pearsonr":pearsonr, "pvalue":pval})
r_pval

In [ ]:
plt.figure(figsize=(6,5))
cor = train_combined.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)

#### TAKEAWAYS

**Features to Target Correlations**
- Square Footage on its own has a MODERATE POSITIVE correlation with Property Value
- Bathroom Count on its own has a MODERATE POSITIVE correlation with Property Value
- Bedroom Count on its own has a VERY LOW POSITIVE correlation with Property Value

**Features to Features Correlations**
- Square Footage and No. of Bathrooms have HIGH POSITIVE correlations with each other
- Square Footage and No. of Bedrooms have MODERATE POSITIVE correlations with each other

**Conclusion**  
Multicollinearity is apparent among the three features. With the Bathroom and Bedroom Count showing High and Moderate Positive Correlations towards the other feature Square Footage, not addressing these correlations might affect fitting the train data to the linear regression model.

To address this, PCA (Principal Component Analysis) will be used since PCA is designed to manage highly correlated variables by "combining" features linearly. Only the highest principal component, i.e., PC1 will be fed into the model.

### B. Feature Engineering

In [ ]:
pca = PCA(n_components=1, copy=True, whiten=False, svd_solver='auto', random_state=123)
pca.fit(scaled_X_train)
pc1 = pca.transform(scaled_X_train)
pc1 = pd.DataFrame(X_pca)

# print(pca.n_components_)
# print(len(X))
print(pca.explained_variance_ratio_)
# print(X[0:5])

In [ ]:
pc1.describe()

## MODEL
PROJECT PLAN -> ACQUIRE -> PRE-PROCESS -> EXPLORE -> **MODEL**

In [ ]:
lmo, lmo_intercept, lmo_coeff = util.generate_linear_model(pc1, y_train)

In [ ]:
lmo.fit(pc1,y)
